# Классификация интентов

## 1. Загрузка библиотек

In [1]:
import random
import json
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle

## 2. Предобработка текста

In [112]:
TOKEN_RE = re.compile(r'[а-яё]+')

with open('BOT_CONFIG_INTENTS.json', encoding='utf-8') as f:
    BOT_CONFIG = json.load(f)
print('Количество интентов:', len(BOT_CONFIG['intents'].keys()))

Количество интентов: 99


In [115]:
def text_cleaning(txt):
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return ' '.join([token for token in all_tokens])

texts = []
y = []
for intent in BOT_CONFIG['intents'].keys():
    for example in BOT_CONFIG['intents'][intent]['examples']:
        texts.append(text_cleaning(example))
        y.append(intent)
print('Количество примеров:', len(texts))
print('Количество меток:', len(y))

Количество примеров: 540
Количество меток: 540


In [116]:
y[-5:]

['philosophy', 'philosophy', 'philosophy', 'philosophy', 'philosophy']

In [117]:
texts[-5:]

['рассуждение',
 'ты чего умничаешь',
 'в чем смысл',
 'умный что ли',
 'мудрая мысль']

## 3. Векторизация

In [118]:
train_texts, test_texts, y_train, y_test = train_test_split(texts, y, random_state=42, test_size=0.2, stratify=y)

In [119]:
vectorizer = CountVectorizer(ngram_range=(1,3), analyzer='char_wb')
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [120]:
vocab = vectorizer.get_feature_names()
print('Количество признаков:', len(vocab))
vocab[-5:]

Количество признаков: 1916


['ятс', 'яты', 'ять', 'ях', 'ях ']

## 4. Моделирование

In [141]:
clf = LogisticRegression(max_iter=10000, random_state=42, C=1.5, l1_ratio=0.5, 
                         penalty='elasticnet', solver='saga').fit(X_train, y_train)
print(f'Точность на Train: {clf.score(X_train, y_train):>10.4f}')
print(f'Точность на Test: {clf.score(X_test, y_test):>11.4f}') 

Точность на Train:     0.9861
Точность на Test:      0.5000


In [140]:
print(f"Метрика F1 на Test: {f1_score(y_test, clf.predict(X_test), average='weighted'):.4f}") 

Метрика F1 на Test: 0.4481


In [129]:
#на всех данных
vectorizer_final = CountVectorizer(ngram_range=(1,3), analyzer='char_wb')
X_full = vectorizer_final.fit_transform(texts)

final_model = LogisticRegression(max_iter=10000, random_state=42, C=1.5, l1_ratio=0.5, 
                         penalty='elasticnet', solver='saga').fit(X_full, y)

In [133]:
with open('intent_model.pkl', 'wb') as f:
    pickle.dump(final_model , f)

In [134]:
with open('intent_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer_final, f)

## 5. Имитация чат бота

In [131]:
def get_intent_by_model(text):
    proba = final_model.predict_proba(vectorizer_final.transform([text]))
    intent = final_model.classes_[proba.argmax()]
    print('Вероятность интента:', proba.max(), f'({intent})')
    if proba.max() >= 0.45:
        return intent

def bot(input_text):
    intent = get_intent_by_model(text_cleaning(input_text))
    if intent:
        return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [132]:
input_text = ''
while input_text != 'stop':
    input_text = input()
    if input_text != 'stop':
        response = bot(input_text)
        if response:
            print(response)

привет
Вероятность интента: 0.6191379130472436 (hello)
Здравствуй, человек
как дела
Вероятность интента: 0.7090670357184188 (howdoyoudo)
хорошо, а ты?
философ
Вероятность интента: 0.561630548125099 (philosophy)
Иногда воспоминания приносят облегчение, а иногда они мука, но мы цепляемся за воспоминания, потому что это все что нам осталось
ты че умный
Вероятность интента: 0.6387338006894999 (philosophy)
Скрывать чувства это не храбрость. Принять боль, пропустить через себя - для этого нужно мужество
умничаешь
Вероятность интента: 0.596663395165071 (philosophy)
Нет больше никакаих жанров, никакаих ярлыков, только люди
какой цвет
Вероятность интента: 0.9633890989660986 (chooseacolor)
красный
какого цвета машина
Вероятность интента: 0.7475656514677066 (chooseacolor)
оранжевый
что почитать
Вероятность интента: 0.7667507845270224 (whattoread)
Шум. Изъян в человеческом суждении
круто
Вероятность интента: 0.11422363539007772 (car)
прикол
Вероятность интента: 0.7819487850992138 (justfun)
))))))
